degrossissage:
contexte: améliorer le taux de succès des rencontres d'une appli de rencontre (Tinder)
elements fondamentaux(en cours) du projet
-   participant
-   interet
question: What makes people interested in each other so as to go on a 2nd_date?

participant est defini initialement par 
    participant:
        -   photo
        -   short bio
        -   liste interets 

le participant fournit 2 retours_info:
 -  un retour_info_self (sur lui-même), de type dating_profile. (A noter le type dating_profile inclu aussi un type d'interet interets personnels)
 -  un retour_info_autre_participant (sur un autre participant), de type interet_autre_participant

Pour fournir "retour_info sur lui-même" l'enquete utilise "recuperer_info_personnelle" sur un participant
et en sortie, fourni  => catégories supplémentaires qui viennent enrichir dating_profile du participant. 
ex enquête_info_personnelle(participant)=>modifie (enrichit) le participant.dating_profile
 -> identifier ces infos personnelles , leur type (Quantitative/qualitative, nominal,ordinal,interval,ratio), dans les datas et voir comment rendre ces catégories mesurables a partir des datas collectées 

Pour fournir "un retour_info_autre_participant" l'enquete utilise "speed_dating" sur 2 participants (avec condition)
et en sortie, fourni  => les mesures des 6 catégories supplémentaires qui qualifient "interet_autre_participant" de chacun des 2 participants
 -> identifier ces 6 categories dans les datas et voir comment rendre ces catégories mesurables a partir des datas collectées

Perimetre: identifier les cas qui correspont à "people interrested in each other" => par ex liste de couples de participants
    Clarification: On inclura un individu interressé par un autre individu sans que cela soit reciproque
    Idée: comparer cela avec individus interressés par un autre individu de maniere reciproque?


But: Fournir les élements mesurables parmis ceux qui compose l'intérêt chez l'autre  
c'est sur ce point qu'on fournira les graphes et qu'on communiquera

Aspect généraux de mise en place du code (en travaux)
il semble qu'il faille des attributs "2nd_date" et "interet_final" de telle sorte que:
    interet_final(participant1,participant2) => True
    interet_final(participant2,participant1) => False

    ex:
        unSpeedDate = speedDate(participant1,participant2,...,autres_attributs) 
        unSpeedDate.interet_final(participant1,participant2) => True
        unSpeedDate.interet_final(participant2,participant1) => False
        unSpeedDate.2nd_date(participant2,participant1) => True si et seulement si 
            unSpeedDate.interet_final() => True pour chaque participant du couple 
pour simplifier on considere que "interet" est l'interet pour un "2nd_date"

Qu'est ce qui constitue un participant?:
    -   dating_profile :
            - photo


qu'est ce qui constitue un speed_date?: (a simplifier )
-   durée=4mn
-   couple de participants(self,participant2)
-   condition= participant1.genre!=participant2.genre
-   interet_final? : True/False
-   interet_autre_participant= 
    {
        "Attractiveness":0,
        "Sincerity":0
        "Intelligence":0
        "Fun":0
        "Ambition":0
        "Shared interests":0
        }

On dispose donc de l'info interet/participant
On peut etudier comment reconstituer les 6 elements qui constituent la liste d'interet 

